## 1. Introduction to Concurrency

In [10]:
import time

# Sequential vs Concurrent
print("=" * 50)
print("SEQUENTIAL EXECUTION")
print("=" * 50)

def task(name, duration):
    print(f"Task {name} started")
    time.sleep(duration)
    print(f"Task {name} completed")

start = time.time()
task("A", 2)
task("B", 2)
task("C", 2)
sequential_time = time.time() - start
print(f"\nSequential Time: {sequential_time:.2f} seconds")

SEQUENTIAL EXECUTION
Task A started
Task A completed
Task B started
Task B completed
Task C started
Task C completed

Sequential Time: 6.00 seconds


## 2. Processes vs Threads

In [11]:
import os
import threading

print("\n" + "=" * 50)
print("PROCESSES vs THREADS")
print("=" * 50)

comparison = """
PROCESS:
- Heavyweight: Each process has its own memory space
- Isolated: Cannot directly share data
- True parallelism: Can run on multiple CPU cores
- Slower to create and switch between
- More secure (isolated memory)

THREAD:
- Lightweight: Shares memory space with parent process
- Shared resources: Easy to share data via shared memory
- Limited by GIL: Only one thread executes at a time
- Faster to create and switch between
- Less secure (shared memory can have race conditions)

CURRENT PROCESS:
"""
print(comparison)
print(f"Process ID: {os.getpid()}")
print(f"Main Thread ID: {threading.current_thread().ident}")
print(f"Active Threads: {threading.active_count()}")


PROCESSES vs THREADS

PROCESS:
- Heavyweight: Each process has its own memory space
- Isolated: Cannot directly share data
- True parallelism: Can run on multiple CPU cores
- Slower to create and switch between
- More secure (isolated memory)

THREAD:
- Lightweight: Shares memory space with parent process
- Shared resources: Easy to share data via shared memory
- Limited by GIL: Only one thread executes at a time
- Faster to create and switch between
- Less secure (shared memory can have race conditions)

CURRENT PROCESS:

Process ID: 30244
Main Thread ID: 29584
Active Threads: 6


## 3. The Global Interpreter Lock (GIL)

In [12]:
print("\n" + "=" * 50)
print("GLOBAL INTERPRETER LOCK (GIL)")
print("=" * 50)

gil_explanation = """
What is GIL?
- Mutex that protects Python objects in CPython
- Only one thread can execute Python bytecode at a time
- Even on multi-core systems

Impact:
- Threads don't provide true parallelism for CPU-bound tasks
- Threads ARE useful for I/O-bound tasks (network, file)
- Processes bypass GIL by using separate Python interpreters

When to use what:
- I/O-bound (network, file): Use THREADS (faster, easier)
- CPU-bound (calculations): Use PROCESSES (true parallelism)
- Mixed: Use PROCESS POOL for CPU + THREAD POOL for I/O
"""
print(gil_explanation)


GLOBAL INTERPRETER LOCK (GIL)

What is GIL?
- Mutex that protects Python objects in CPython
- Only one thread can execute Python bytecode at a time
- Even on multi-core systems

Impact:
- Threads don't provide true parallelism for CPU-bound tasks
- Threads ARE useful for I/O-bound tasks (network, file)
- Processes bypass GIL by using separate Python interpreters

When to use what:
- I/O-bound (network, file): Use THREADS (faster, easier)
- CPU-bound (calculations): Use PROCESSES (true parallelism)
- Mixed: Use PROCESS POOL for CPU + THREAD POOL for I/O



## 4. Threading Basics

In [13]:
import threading
import time

print("\n" + "=" * 50)
print("THREADING EXAMPLE")
print("=" * 50)

def worker(name, duration):
    print(f"[{threading.current_thread().name}] Starting task {name}")
    time.sleep(duration)
    print(f"[{threading.current_thread().name}] Completed task {name}")

# Create threads
threads = []
start = time.time()

for i in range(3):
    t = threading.Thread(target=worker, args=(f"Task-{i+1}", 2))
    threads.append(t)
    t.start()

# Wait for all threads
for t in threads:
    t.join()

threading_time = time.time() - start
print(f"\nThreading Time: {threading_time:.2f} seconds")
print(f"Improvement: {sequential_time / threading_time:.1f}x faster")


THREADING EXAMPLE
[Thread-16 (worker)] Starting task Task-1
[Thread-17 (worker)] Starting task Task-2
[Thread-18 (worker)] Starting task Task-3
[Thread-16 (worker)] Completed task Task-1[Thread-17 (worker)] Completed task Task-2
[Thread-18 (worker)] Completed task Task-3


Threading Time: 2.06 seconds
Improvement: 2.9x faster


## 5. Multiprocessing Basics

In [14]:
import multiprocessing
import time

print("\n" + "=" * 50)
print("MULTIPROCESSING EXAMPLE")
print("=" * 50)

def process_worker(name, duration):
    print(f"[Process {multiprocessing.current_process().name}] Starting {name}")
    time.sleep(duration)
    print(f"[Process {multiprocessing.current_process().name}] Completed {name}")

if __name__ == '__main__':
    processes = []
    start = time.time()
    
    for i in range(3):
        p = multiprocessing.Process(
            target=process_worker,
            args=(f"Task-{i+1}", 2)
        )
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
    
    mp_time = time.time() - start
    print(f"\nMultiprocessing Time: {mp_time:.2f} seconds")


MULTIPROCESSING EXAMPLE

Multiprocessing Time: 0.37 seconds


## 6. CPU-Bound vs I/O-Bound Tasks

In [15]:
import time
import threading
import multiprocessing

print("\n" + "=" * 50)
print("CPU-BOUND vs I/O-BOUND TASKS")
print("=" * 50)

# CPU-Bound Task
def cpu_bound_task(n):
    count = 0
    for i in range(n):
        count += i
    return count

# I/O-Bound Task
def io_bound_task(duration):
    time.sleep(duration)  # Simulates I/O
    return f"I/O completed in {duration}s"

# CPU-Bound Characteristics
print("""
CPU-BOUND TASKS:
- Intensive calculations
- Image processing
- Data compression
- Mathematics operations
BEST FOR: Multiprocessing (bypass GIL)

I/O-BOUND TASKS:
- Network requests
- File operations
- Database queries
- API calls
BEST FOR: Threading (simple, efficient)
""")

# Quick test
print("Testing CPU-bound with 10M operations...")
start = time.time()
result = cpu_bound_task(10_000_000)
print(f"Time: {time.time() - start:.2f}s")

print("\nTesting I/O-bound with 2s sleep...")
start = time.time()
result = io_bound_task(2)
print(f"Time: {time.time() - start:.2f}s")


CPU-BOUND vs I/O-BOUND TASKS

CPU-BOUND TASKS:
- Intensive calculations
- Image processing
- Data compression
- Mathematics operations
BEST FOR: Multiprocessing (bypass GIL)

I/O-BOUND TASKS:
- Network requests
- File operations
- Database queries
- API calls
BEST FOR: Threading (simple, efficient)

Testing CPU-bound with 10M operations...
Time: 0.38s

Testing I/O-bound with 2s sleep...
Time: 2.00s


## 7. Thread Safety and Race Conditions

In [16]:
import threading
import time

print("\n" + "=" * 50)
print("THREAD SAFETY & RACE CONDITIONS")
print("=" * 50)

# UNSAFE: Race condition
print("\nUNSAFE VERSION (Race Condition):")
counter = 0
lock = threading.Lock()

def unsafe_increment():
    global counter
    for _ in range(100_000):
        counter += 1

threads = [threading.Thread(target=unsafe_increment) for _ in range(5)]
start = time.time()
for t in threads:
    t.start()
for t in threads:
    t.join()
print(f"Counter value: {counter} (Expected: 500000)")

# SAFE: Using lock
print("\nSAFE VERSION (With Lock):")
counter = 0

def safe_increment():
    global counter
    for _ in range(100_000):
        with lock:
            counter += 1

threads = [threading.Thread(target=safe_increment) for _ in range(5)]
for t in threads:
    t.start()
for t in threads:
    t.join()
print(f"Counter value: {counter} (Expected: 500000)")


THREAD SAFETY & RACE CONDITIONS

UNSAFE VERSION (Race Condition):
Counter value: 500000 (Expected: 500000)

SAFE VERSION (With Lock):
Counter value: 500000 (Expected: 500000)


## 8. Synchronization Primitives

In [17]:
import threading

print("\n" + "=" * 50)
print("SYNCHRONIZATION PRIMITIVES")
print("=" * 50)

sync_info = """
LOCK (Mutex):
- Prevents multiple threads from accessing shared resource
- One thread at a time
lock = threading.Lock()
with lock:
    # Critical section
    shared_resource += 1

EVENT:
- One thread signals, others wait
event = threading.Event()
event.wait()  # Wait for signal
event.set()   # Signal

CONDITION:
- Threads wait for specific condition
cond = threading.Condition()
with cond:
    cond.wait()  # Wait for condition
    cond.notify()  # Signal

SEMAPHORE:
- Limits number of threads accessing resource
sem = threading.Semaphore(3)  # Max 3 threads
with sem:
    # Only 3 threads can be here at once

QUEUE:
- Thread-safe data structure for passing messages
from queue import Queue
q = Queue()
q.put(item)  # Add item
item = q.get()  # Get item (blocks if empty)
"""
print(sync_info)


SYNCHRONIZATION PRIMITIVES

LOCK (Mutex):
- Prevents multiple threads from accessing shared resource
- One thread at a time
lock = threading.Lock()
with lock:
    # Critical section
    shared_resource += 1

EVENT:
- One thread signals, others wait
event = threading.Event()
event.wait()  # Wait for signal
event.set()   # Signal

CONDITION:
- Threads wait for specific condition
cond = threading.Condition()
with cond:
    cond.wait()  # Wait for condition
    cond.notify()  # Signal

SEMAPHORE:
- Limits number of threads accessing resource
sem = threading.Semaphore(3)  # Max 3 threads
with sem:
    # Only 3 threads can be here at once

QUEUE:
- Thread-safe data structure for passing messages
from queue import Queue
q = Queue()
q.put(item)  # Add item
item = q.get()  # Get item (blocks if empty)



## 9. Key Concepts Summary

In [18]:
print("\n" + "=" * 50)
print("SUMMARY: WHEN TO USE WHAT")
print("=" * 50)

summary = """
✓ USE THREADING WHEN:
  • I/O-bound operations (network, files, databases)
  • Simple concurrent tasks
  • Quick implementation needed
  • Shared state management is simple
  • Example: Web scraping, API calls, file downloads

✓ USE MULTIPROCESSING WHEN:
  • CPU-bound operations (calculations, processing)
  • Need true parallelism on multiple cores
  • Tasks are independent
  • Heavy computational work
  • Example: Image processing, data analysis, simulations

✓ USE ASYNC/AWAIT WHEN:
  • Many I/O operations (thousands of connections)
  • Need fine-grained control
  • Building async frameworks
  • Example: Web servers, real-time applications

⚠ THINGS TO REMEMBER:
  • GIL prevents true parallelism with threads
  • Threads share memory (watch for race conditions)
  • Processes have separate memory (slower communication)
  • Use queues for thread-safe communication
  • Always use locks for shared data
  • Join threads before program exit
"""
print(summary)


SUMMARY: WHEN TO USE WHAT

✓ USE THREADING WHEN:
  • I/O-bound operations (network, files, databases)
  • Simple concurrent tasks
  • Quick implementation needed
  • Shared state management is simple
  • Example: Web scraping, API calls, file downloads

✓ USE MULTIPROCESSING WHEN:
  • CPU-bound operations (calculations, processing)
  • Need true parallelism on multiple cores
  • Tasks are independent
  • Heavy computational work
  • Example: Image processing, data analysis, simulations

✓ USE ASYNC/AWAIT WHEN:
  • Many I/O operations (thousands of connections)
  • Need fine-grained control
  • Building async frameworks
  • Example: Web servers, real-time applications

⚠ THINGS TO REMEMBER:
  • GIL prevents true parallelism with threads
  • Threads share memory (watch for race conditions)
  • Processes have separate memory (slower communication)
  • Use queues for thread-safe communication
  • Always use locks for shared data
  • Join threads before program exit

